<a href="https://colab.research.google.com/github/khanhlvg/tflite_raspberry_pi/blob/main/object_detection/Train_custom_model_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Train a custom object detection model with TensorFlow Lite Model Maker

In this colab notebook, you'll learn how to use the [TensorFlow Lite Model Maker](https://www.tensorflow.org/lite/guide/model_maker) to train a custom object detection model to detect Android figurines and how to put the model on a Raspberry Pi.

The Model Maker library uses *transfer learning* to simplify the process of training a TensorFlow Lite model using a custom dataset. Retraining a TensorFlow Lite model with your own custom dataset reduces the amount of training data required and will shorten the training time.


## Preparation

### Install the required packages
Start by installing the required packages, including the Model Maker package from the [GitHub repo](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker) and the pycocotools library you'll use for evaluation.

In [1]:
!pip install -q tflite-model-maker
!pip install -q tflite-support

     |████████████████████████████████| 616 kB 7.0 MB/s 
     |████████████████████████████████| 87 kB 7.5 MB/s 
     |████████████████████████████████| 77 kB 6.4 MB/s 
     |████████████████████████████████| 1.1 MB 57.4 MB/s 
     |████████████████████████████████| 213 kB 63.7 MB/s 
     |████████████████████████████████| 1.2 MB 58.1 MB/s 
     |████████████████████████████████| 120 kB 69.8 MB/s 
     |████████████████████████████████| 1.1 MB 68.2 MB/s 
     |████████████████████████████████| 3.4 MB 48.4 MB/s 
     |████████████████████████████████| 6.4 MB 53.9 MB/s 
     |████████████████████████████████| 840 kB 52.9 MB/s 
     |████████████████████████████████| 596 kB 66.1 MB/s 
     |████████████████████████████████| 25.3 MB 1.5 MB/s 
     |████████████████████████████████| 352 kB 57.4 MB/s 
     |████████████████████████████████| 47.7 MB 108 kB/s 
     |████████████████████████████████| 99 kB 12.3 MB/s 
     |████████████████████████████████| 210 kB 69.4 MB/s 


Import the required packages.

In [2]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

## Train the object detection model

### Step 1: Load the dataset

* Images in `train_data` is used to train the custom object detection model.
* Images in `val_data` is used to check if the model can generalize well to new images that it hasn't seen before.

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'Colab Notebooks'   freedomtech.zip   yolov3


In [4]:
!unzip /mydrive/freedomtech.zip

Archive:  /mydrive/freedomtech.zip
   creating: freedomtech/train/
  inflating: freedomtech/train/IMG_20220118_073141.jpg  
  inflating: freedomtech/train/IMG_20220117_184918.jpg  
  inflating: freedomtech/train/IMG_20220117_184607.jpg  
  inflating: freedomtech/train/IMG_20220118_072843.jpg  
  inflating: freedomtech/train/IMG_20220117_184539.jpg  
  inflating: freedomtech/train/IMG_20220118_072632.jpg  
  inflating: freedomtech/train/IMG_20220117_184940.jpg  
  inflating: freedomtech/train/IMG_20220117_185008.jpg  
  inflating: freedomtech/train/IMG_20220118_073137.xml  
  inflating: freedomtech/train/IMG_20220117_184913.jpg  
  inflating: freedomtech/train/IMG_20220117_184247.jpg  
  inflating: freedomtech/train/IMG_20220118_073221.jpg  
  inflating: freedomtech/train/IMG_20220118_072802.jpg  
  inflating: freedomtech/train/IMG_20220118_073115.xml  
  inflating: freedomtech/train/IMG_20220117_184947.xml  
  inflating: freedomtech/train/IMG_20220117_184502.jpg  
  inflating: freedomt

In [5]:
train_data = object_detector.DataLoader.from_pascal_voc(
    'freedomtech/train',
    'freedomtech/train',
    ['esp8266', 'pico']
)

val_data = object_detector.DataLoader.from_pascal_voc(
    'freedomtech/validate',
    'freedomtech/validate',
    ['esp8266', 'pico']
)

### Step 2: Select a model architecture

EfficientDet-Lite[0-4] are a family of mobile/IoT-friendly object detection models derived from the [EfficientDet](https://arxiv.org/abs/1911.09070) architecture.

Here is the performance of each EfficientDet-Lite models compared to each others.

| Model architecture | Size(MB)* | Latency(ms)** | Average Precision*** |
|--------------------|-----------|---------------|----------------------|
| EfficientDet-Lite0 | 4.4       | 146           | 25.69%               |
| EfficientDet-Lite1 | 5.8       | 259           | 30.55%               |
| EfficientDet-Lite2 | 7.2       | 396           | 33.97%               |
| EfficientDet-Lite3 | 11.4      | 716           | 37.70%               |
| EfficientDet-Lite4 | 19.9      | 1886          | 41.96%               |

<i> * Size of the integer quantized models. <br/>
** Latency measured on Raspberry Pi 4 using 4 threads on CPU. <br/>
*** Average Precision is the mAP (mean Average Precision) on the COCO 2017 validation dataset.
</i>

In this notebook, we use EfficientDet-Lite0 to train our model. You can choose other model architectures depending on whether speed or accuracy is more important to you.

In [6]:
spec = model_spec.get('efficientdet_lite0')

### Step 3: Train the TensorFlow model with the training data.

* Set `epochs = 20`, which means it will go through the training dataset 20 times. You can look at the validation accuracy during training and stop when you see validation loss (`val_loss`) stop decreasing to avoid overfitting.
* Set `batch_size = 4` here so you will see that it takes 15 steps to go through the 62 images in the training dataset.
* Set `train_whole_model=True` to fine-tune the whole model instead of just training the head layer to improve accuracy. The trade-off is that it may take longer to train the model.

In [7]:
model = object_detector.create(train_data, model_spec=spec, batch_size=4, train_whole_model=True, epochs=20, validation_data=val_data)

Epoch 1/20
19/19 [==============================] - 121s 4s/step - det_loss: 1.7226 - cls_loss: 1.1302 - box_loss: 0.0118 - reg_l2_loss: 0.0630 - loss: 1.7856 - learning_rate: 0.0065 - gradient_norm: 1.8107 - val_det_loss: 1.6424 - val_cls_loss: 1.0987 - val_box_loss: 0.0109 - val_reg_l2_loss: 0.0630 - val_loss: 1.7054
Epoch 2/20
19/19 [==============================] - 73s 4s/step - det_loss: 1.6114 - cls_loss: 1.0738 - box_loss: 0.0108 - reg_l2_loss: 0.0630 - loss: 1.6743 - learning_rate: 0.0049 - gradient_norm: 2.1609 - val_det_loss: 1.5479 - val_cls_loss: 1.0054 - val_box_loss: 0.0109 - val_reg_l2_loss: 0.0630 - val_loss: 1.6109
Epoch 3/20
19/19 [==============================] - 74s 4s/step - det_loss: 1.4470 - cls_loss: 0.9480 - box_loss: 0.0100 - reg_l2_loss: 0.0630 - loss: 1.5100 - learning_rate: 0.0048 - gradient_norm: 2.5148 - val_det_loss: 1.4553 - val_cls_loss: 0.9533 - val_box_loss: 0.0100 - val_reg_l2_loss: 0.0630 - val_loss: 1.5183
Epoch 4/20
19/19 [=====================

### Step 4. Evaluate the model with the validation data.

After training the object detection model using the images in the training dataset, use the 10 images in the validation dataset to evaluate how the model performs against new data it has never seen before.

As the default batch size is 64, it will take 1 step to go through the 10 images in the validation dataset.

The evaluation metrics are same as [COCO](https://cocodataset.org/#detection-eval).

In [8]:
model.evaluate(val_data)

2/2 [==============================] - 45s 3s/step



{'AP': 0.3995251,
 'AP50': 0.6755185,
 'AP75': 0.47726184,
 'AP_/esp8266': 0.39452982,
 'AP_/pico': 0.40452036,
 'APl': 0.3995251,
 'APm': -1.0,
 'APs': -1.0,
 'ARl': 0.5041818,
 'ARm': -1.0,
 'ARmax1': 0.44390908,
 'ARmax10': 0.48327273,
 'ARmax100': 0.5041818,
 'ARs': -1.0}

### Step 5: Export as a TensorFlow Lite model.

Export the trained object detection model to the TensorFlow Lite format by specifying which folder you want to export the quantized model to. The default post-training quantization technique is [full integer quantization](https://www.tensorflow.org/lite/performance/post_training_integer_quant). This allows the TensorFlow Lite model to be smaller, run faster on Raspberry Pi CPU and also compatible with the Google Coral EdgeTPU.

In [9]:
model.export(export_dir='.', tflite_filename='android.tflite')

### Step 6:  Evaluate the TensorFlow Lite model.

Several factors can affect the model accuracy when exporting to TFLite:
* [Quantization](https://www.tensorflow.org/lite/performance/model_optimization) helps shrinking the model size by 4 times at the expense of some accuracy drop.
* The original TensorFlow model uses per-class [non-max supression (NMS)](https://www.coursera.org/lecture/convolutional-neural-networks/non-max-suppression-dvrjH) for post-processing, while the TFLite model uses global NMS that's much faster but less accurate.
Keras outputs maximum 100 detections while tflite outputs maximum 25 detections.

Therefore you'll have to evaluate the exported TFLite model and compare its accuracy with the original TensorFlow model.

Before compiling the `.tflite` file for the Edge TPU, it's important to consider whether your model will fit into the Edge TPU memory. 

The Edge TPU has approximately 8 MB of SRAM for [caching model paramaters](https://coral.ai/docs/edgetpu/compiler/#parameter-data-caching), so any model close to or over 8 MB will not fit onto the Edge TPU memory. That means the inference times are longer, because some model parameters must be fetched from the host system memory.

One way to elimiate the extra latency is to use [model pipelining](https://coral.ai/docs/edgetpu/pipeline/), which splits the model into segments that can run on separate Edge TPUs in series. This can significantly reduce the latency for big models.

The following table provides recommendations for the number of Edge TPUs to use with each EfficientDet-Lite model.

| Model architecture | Minimum TPUs | Recommended TPUs
|--------------------|-------|-------|
| EfficientDet-Lite0 | 1     | 1     |
| EfficientDet-Lite1 | 1     | 1     |
| EfficientDet-Lite2 | 1     | 2     |
| EfficientDet-Lite3 | 2     | 2     |
| EfficientDet-Lite4 | 2     | 3     |

If you need extra Edge TPUs for your model, then update `NUMBER_OF_TPUS` here:

Finally, we'll copy the metadata, including the label file, from the original TensorFlow Lite model to the EdgeTPU model.